<a href="https://colab.research.google.com/github/ShubhamNapNap/djs-gdg-tasks/blob/main/Deploy_Ml_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 200)
df = pd.read_csv("f1_dnf.csv")
df = df[[
    'resultId', 'year', 'round', 'grid', 'positionOrder', 'points', 'laps', 'dob', 'date', 'target_finish',
    'driverRef', 'forename', 'surname', 'nationality_x',
    'constructorRef', 'name', 'nationality_y',
    'circuitRef', 'name_y', 'location', 'country', 'lat', 'lng', 'alt'
]].copy()
df['dob'] = pd.to_datetime(df['dob'])
df['date'] = pd.to_datetime(df['date'])
df['driver_age_at_race'] = (df['date'] - df['dob']).dt.days / 365.25
df['driver_name'] = df['forename'] + ' ' + df['surname']
df = df.drop(columns=['forename', 'surname'])
df = df.loc[~df.duplicated(subset=["year", "round", "driverRef", "constructorRef"])]\
.reset_index(drop=True).copy()
df.head()

,resultId,year,round,grid,positionOrder,points,laps,dob,date,target_finish,driverRef,nationality_x,constructorRef,name,nationality_y,circuitRef,name_y,location,country,lat,lng,alt,driver_age_at_race,driver_name
0,2460,2002,13,11,4,3.0,77.0,1979-10-17,2002-08-18,1,raikkonen,Finnish,mclaren,Hungaroring,British,hungaroring,McLaren,Budapest,Hungary,47.5789,19.24860,264,22.836413,Kimi Räikkönen
1,11565,1981,1,23,21,0.0,16.0,1946-05-04,1981-03-15,0,watson,British,mclaren,Long Beach,British,long_beach,McLaren,California,USA,33.7651,-118.18900,12,34.863792,John Watson
2,18661,1958,8,0,26,0.0,0.0,1930-03-11,1958-08-03,0,ruttman,American,maserati,Nürburgring,Italian,nurburgring,Maserati,Nürburg,Germany,50.3356,6.94750,578,28.396988,Troy Ruttman
3,25121,2021,8,19,16,0.0,69.0,1999-03-22,2021-06-27,0,mick_schumacher,German,haas,Red Bull Ring,American,red_bull_ring,Haas F1 Team,Spielberg,Austria,47.2197,14.76470,678,22.266940,Mick Schumacher
4,8863,1988,12,0,30,0.0,0.0,1963-05-12,1988-09-11,0,modena,Italian,eurobrun,Autodromo Nazionale di Monza,Italian,monza,Euro Brun,Monza,Italy,45.6156,9.28111,162,25.336071,Stefano Modena


In [ ]:
df = df.drop(columns=['resultId', 'dob','positionOrder', 'date', 'driver_name','name', 'name_y','location','circuitRef', 'country', 'alt','nationality_y','nationality_x','laps','points'])
df.head()

,year,round,grid,target_finish,driverRef,constructorRef,lat,lng,driver_age_at_race
0,2002,13,11,1,raikkonen,mclaren,47.5789,19.24860,22.836413
1,1981,1,23,0,watson,mclaren,33.7651,-118.18900,34.863792
2,1958,8,0,0,ruttman,maserati,50.3356,6.94750,28.396988
3,2021,8,19,0,mick_schumacher,haas,47.2197,14.76470,22.266940
4,1988,12,0,0,modena,eurobrun,45.6156,9.28111,25.336071


In [ ]:
df.isna().sum()

,0
year,0
round,0
grid,0
target_finish,0
driverRef,0
constructorRef,0
lat,0
lng,0
driver_age_at_race,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

le_constructor = LabelEncoder()
df['constructorRef_encoded'] = le_constructor.fit_transform(df['constructorRef'])

le_driver = LabelEncoder()
df['driverRef_encoded'] = le_driver.fit_transform(df['driverRef'])
df.drop(columns=['constructorRef', 'driverRef'], inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
x = df.drop(columns=['target_finish'])
y = df['target_finish']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
model = RandomForestClassifier(n_estimators=700, max_depth=12, random_state=1)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=12, n_estimators=700, random_state=1)

In [ ]:
st.title('Formula 1 Finish Prediction')
grid = st.sidebar.number_input("Grid Position", min_value=1, max_value=20, value=1)
position_order = st.sidebar.number_input("Position Order", min_value=1, max_value=20, value=1)
driver_age = st.sidebar.number_input("Driver Age at Race", min_value=18, max_value=50, value=25)
constructor = st.sidebar.selectbox("Constructor", options=df['constructorRef_encoded'].unique())
driver = st.sidebar.selectbox("Driver", options=df['driverRef_encoded'].unique())
input_data = np.array([[grid, position_order, driver_age, constructor, driver]])

In [ ]:
if st.sidebar.button("Predict"):
    prediction = model.predict(input_data)
    prediction_proba = model.predict_proba(input_data)[:, 1]

    if prediction[0] == 1:
        st.write("DNF")
    else:
        st.write("Finished")


In [ ]:
#y_pred = model.predict(X_test)
#y_pred_proba = model.predict_proba(X_test)[:, 1]

In [ ]:
'''from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)'''


In [ ]:
'''print(f'Accuracy: {accuracy_score(y_test, y_pred):.3f}')
print(f'Precision: {precision_score(y_test, y_pred):.3f}')
print(f'Recall: {recall_score(y_test, y_pred):.3f}')
print(f'F1 Score: {f1_score(y_test, y_pred):.3f}')
print(f'ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.3f}')

print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred))

print('\nClassification Report:')
print(classification_report(y_test, y_pred))'''

Accuracy: 0.806
Precision: 0.723
Recall: 0.535
F1 Score: 0.615
ROC-AUC: 0.851

Confusion Matrix:
[[1301  119]
 [ 269  310]]

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1420
           1       0.72      0.54      0.62       579

    accuracy                           0.81      1999
   macro avg       0.78      0.73      0.74      1999
weighted avg       0.80      0.81      0.80      1999

